このNotebookは1-5章のスクリプトの実行が面倒であったため、Jupyter Notebook上から実行できるように作成しました。

`open3d==0.16.1`で動作確認しています。

`open3d==0.16.０`以降から使えるようになったNotebook内に点群を描画できる`draw_plotly`を極力使いながら、本に掲載されている画像とアウトプットの質感が異なってしまう場合には`draw_geometries`を使用しています。

## ファイルの中身の確認

In [ ]:
import struct

filename = './3rdparty/Open3D/examples/test_data/bathtub_0154.ply'

with open(filename, 'rb') as f:
    # read header
    while True:
        line = f.readline()
        print (line)
        if b'end_header' in line:
            break
        if b'vertex ' in line:
            vnum = int(line.split(b' ')[-1]) # num of vertices
        if b'face ' in line:
            fnum = int(line.split(b' ')[-1]) # num of faces

    # read vertices
    for i in range(vnum):
        for j in range(3):
            print (struct.unpack('f', f.read(4))[0], end=' ')
        print ("")

    # read faces
    for i in range(fnum):
        n = struct.unpack('B', f.read(1))[0]
        for j in range(n):
            print (struct.unpack('i', f.read(4))[0], end=' ')
        print ("")

## 描画

In [ ]:
import sys
import numpy as np
import open3d as o3d
from open3d.visualization import draw_plotly

filename = './3rdparty/Open3D/examples/test_data/bathtub_0154.ply'
#pcd = o3d.io.read_point_cloud(filename)
mesh = o3d.io.read_triangle_mesh(filename)

#draw_plotly([pcd])
draw_plotly([mesh])


In [ ]:
import sys
import numpy as np
import open3d as o3d
from open3d.visualization import draw_plotly

filename = './3rdparty/Open3D/examples/test_data/fragment.ply'
pcd = o3d.io.read_point_cloud(filename)
draw_plotly([pcd])

## 回転・並進・スケール変換

In [ ]:
%%python

import numpy as np
import open3d as o3d
import copy

# Create axes
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()

# Rotate
R = o3d.geometry.get_rotation_matrix_from_yxz([np.pi/3, 0, 0])
print ("R:",np.round(R,7))
R = o3d.geometry.get_rotation_matrix_from_axis_angle([0, np.pi/3, 0])
print ("R:",np.round(R,7))
R = o3d.geometry.get_rotation_matrix_from_quaternion([np.cos(np.pi/6), 0, np.sin(np.pi/6), 0])
print ("R:",np.round(R,7))
mesh_r = copy.deepcopy(mesh)
mesh_r.rotate(R, center=[0,0,0])

# Translate
t = [0.5, 0.7, 1]
mesh_t = copy.deepcopy(mesh_r).translate(t)
print ("Type q to continue.")
o3d.visualization.draw_geometries([mesh, mesh_t])

# Rotate and translate
T = np.eye(4)
T[:3,:3] = R
T[:3,3] = t
mesh_t = copy.deepcopy(mesh).transform(T)
print ("Type q to continue.")
o3d.visualization.draw_geometries([mesh, mesh_t])

# Scale
mesh_s = copy.deepcopy(mesh_t)
mesh_s.scale(0.5, center=mesh_s.get_center())
print ("Type q to exit.")
o3d.visualization.draw_geometries([mesh, mesh_s])

## サンプリング

### 等間隔サンプリングによるボクセル化

In [ ]:
import sys
import open3d as o3d

filename = './3rdparty/Open3D/examples/test_data/fragment.ply'
s = 0.03  # 1辺の大きさ

print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

o3d.visualization.draw_plotly([pcd], zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

downpcd = pcd.voxel_down_sample(voxel_size=s)
print(downpcd)

o3d.visualization.draw_plotly([downpcd], zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

### Farthest Point Sampling(FPS)

In [ ]:
import sys
import numpy as np
import open3d as o3d


def l2_norm(a, b):
    return ((a - b) ** 2).sum(axis=1)


def farthest_point_sampling(pcd, k, metrics=l2_norm):
    indices = np.zeros(k, dtype=np.int32)
    points = np.asarray(pcd.points)
    distances = np.zeros((k, points.shape[0]), dtype=np.float32)
    indices[0] = np.random.randint(len(points))
    farthest_point = points[indices[0]]
    min_distances = metrics(farthest_point, points)
    distances[0, :] = min_distances
    for i in range(1, k):
        indices[i] = np.argmax(min_distances)
        farthest_point = points[indices[i]]
        distances[i, :] = metrics(farthest_point, points)
        min_distances = np.minimum(min_distances, distances[i, :])
    pcd = pcd.select_by_index(indices)
    return pcd


# main
filename =  './3rdparty/Open3D/examples/test_data/fragment.ply'
k = 1000  # サンプリングする点の数

print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

o3d.visualization.draw_plotly([pcd], zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

downpcd = farthest_point_sampling(pcd, k)
print(downpcd)

o3d.visualization.draw_plotly([downpcd], zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])


In [ ]:
import sys
import open3d as o3d


# main
filename =  './3rdparty/Open3D/examples/test_data/fragment.ply'
k = 1000  # サンプリングする点の数

print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

o3d.visualization.draw_plotly([pcd], zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

downpcd = pcd.farthest_point_down_sample(num_samples=k)
print(downpcd)

o3d.visualization.draw_plotly([downpcd], zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])


### Poisson Disk Sampling(PDS)

In [ ]:
import sys
import open3d as o3d

filename = './3rdparty/Open3D/examples/test_data/knot.ply'
k = 500

print("Loading a triangle mesh from", filename)
mesh = o3d.io.read_triangle_mesh(filename)
print(mesh)

o3d.visualization.draw_plotly([mesh], mesh_show_wireframe=True)

downpcd = mesh.sample_points_poisson_disk(number_of_points=k)
print(downpcd)

o3d.visualization.draw_plotly([downpcd])

## 外れ値除去

In [ ]:
%%python

import sys
import open3d as o3d

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                      zoom=0.3412,
                                      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024]
                                 )

filename = './3rdparty/Open3D/examples/test_data/fragment.ply'
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

print("Statistical oulier removal")
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
display_inlier_outlier(pcd, ind)

print("Radius oulier removal")
cl, ind = pcd.remove_radius_outlier(nb_points=16, radius=0.02)
display_inlier_outlier(pcd, ind)

## 法線推定

In [ ]:
%%python

import sys
import open3d as o3d
import numpy as np

filename = './3rdparty/Open3D/examples/test_data/knot.ply'
print("Loading a point cloud from", filename)
mesh = o3d.io.read_triangle_mesh(filename)
print(mesh)
o3d.visualization.draw_geometries([mesh])

pcd = o3d.geometry.PointCloud()
pcd.points = mesh.vertices
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=10.0, max_nn=10))

print(np.asarray(pcd.normals))
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

mesh.compute_vertex_normals()

print(np.asarray(mesh.triangle_normals))
o3d.visualization.draw_geometries([mesh])

## 特徴点抽出

データを事前にダウンロードする
```shell
wget http://graphics.stanford.edu/pub/3Dscanrep/bunny.tar.gz
tar xvfz bunny.tar.gz    
mv bunny/reconstruction/bun_zipper.ply Bunny.ply
rm -r {bunny,bunny.tar.gz}
```

### Harris3D

以下のコードはUbuntuではうまく描画できたがMac(M1)ではなぜかabortとなり、うまく描画できなかった。

In [ ]:
import sys
import open3d as o3d
import numpy as np


# This function is only used to make the keypoints look better on the rendering
def keypoints_to_spheres(keypoints):
    spheres = o3d.geometry.TriangleMesh()
    for keypoint in keypoints.points:
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.001)
        sphere.translate(keypoint)
        spheres += sphere
    spheres.paint_uniform_color([1.0, 0.75, 0.0])
    return spheres


def compute_harris3d_keypoints( pcd, radius=0.01, max_nn=10, threshold=0.001 ):
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    harris = np.zeros( len(np.asarray(pcd.points)) )
    is_active = np.zeros( len(np.asarray(pcd.points)), dtype=bool )

    # Compute Harris response
    for i in range( len(np.asarray(pcd.points)) ):
        [num_nn, inds, _] = pcd_tree.search_knn_vector_3d(pcd.points[i], max_nn)
        pcd_normals = pcd.select_by_index(inds)
        pcd_normals.points = pcd_normals.normals
        [_, covar] = pcd_normals.compute_mean_and_covariance()
        harris[ i ] = np.linalg.det( covar ) / np.trace( covar )
        if (harris[ i ] > threshold):
            is_active[ i ] = True

    # NMS
    for i in range( len(np.asarray(pcd.points)) ):
        if is_active[ i ]:
            [num_nn, inds, _] = pcd_tree.search_knn_vector_3d(pcd.points[i], max_nn)
            inds.pop( harris[inds].argmax() )
            is_active[ inds ] = False

    keypoints = pcd.select_by_index(np.where(is_active)[0])
    return keypoints


# main
filename = './3rdparty/Open3D/examples/test_data/Bunny.ply'
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

keypoints = compute_harris3d_keypoints(pcd)
print(keypoints)

pcd.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_geometries([keypoints_to_spheres(keypoints), pcd])

### Intrinsic Shape Signature(ISS)

以下のコードはUbuntuではうまく描画できたがMac(M1)ではなぜかabortとなり、うまく描画できなかった。

In [ ]:
import sys
import open3d as o3d


# This function is only used to make the keypoints look better on the rendering
def keypoints_to_spheres(keypoints):
    spheres = o3d.geometry.TriangleMesh()
    for keypoint in keypoints.points:
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.001)
        sphere.translate(keypoint)
        spheres += sphere
    spheres.paint_uniform_color([1.0, 0.75, 0.0])
    return spheres


filename = './3rdparty/Open3D/examples/test_data/Bunny.ply'
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

keypoints = \
o3d.geometry.keypoint.compute_iss_keypoints(pcd,
                                            salient_radius=0.005,
                                            non_max_radius=0.005,
                                            gamma_21=0.5,
                                            gamma_32=0.5)
print(keypoints)

pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=10))
pcd.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_geometries([keypoints_to_spheres(keypoints), pcd])


## 局所特徴量

### Fast Point Feature Histgram(FPFH)

多くの場合、特徴点抽出した後に特徴点に対してのみ特徴量を計算する。

In [ ]:
import sys
import open3d as o3d

filename = './3rdparty/Open3D/examples/test_data/Bunny.ply'
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

pcd.estimate_normals(
    search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=10))

fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd,
    search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100))

print(fpfh)
print(fpfh.data)

## 点群レジストレーション（位置合わせ）

### 最近傍点の探索

In [ ]:
%%python

import open3d as o3d
import numpy as np
import copy

def dist( p, X ):
    dists = np.linalg.norm(p-X,axis=1) 
    min_dist = min(dists)
    min_idx = np.argmin(dists)
    
    return min_dist, min_idx

# point cloud as sin function
X_x = np.arange(-np.pi,np.pi, 0.1)
X_y = np.sin(X_x)
X_z = np.zeros(X_x.shape)
X = np.vstack([X_x, X_y, X_z]).T

# point p
p = np.array([1.0,0.0,0.0])

# open3d point cloud of X
pcd_X = o3d.geometry.PointCloud()
pcd_X.points = o3d.utility.Vector3dVector(X)
pcd_X.paint_uniform_color([0.5,0.5,0.5])

# open3d point cloud of p
pcd_p = o3d.geometry.PointCloud()
pcd_p.points = o3d.utility.Vector3dVector([p])
pcd_p.paint_uniform_color([0.0,0.0,1.0])

# Create axis
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()

# Visualization
o3d.visualization.draw_geometries([mesh, pcd_X,pcd_p])


min_dist, min_idx = dist(p,X)
np.asarray(pcd_X.colors)[min_idx] = [0.0,1.0,0.0]
print("distance:{}, idx:{}".format(min_dist, min_idx))
o3d.visualization.draw_geometries([mesh,pcd_X,pcd_p])

### Open3Dでのkd-tree

In [ ]:
import open3d as o3d
import numpy as np
import copy

pcd = o3d.io.read_point_cloud("data/bun000.pcd")
pcd.paint_uniform_color([0.5, 0.5, 0.5])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

query = 10000
pcd.colors[query] = [1, 0, 0]

[k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[query], 200)
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]
o3d.visualization.draw_plotly([pcd], width=600, height=400)

query = 20000
pcd.colors[query] = [1, 0, 0]
[k, idx, d] = pcd_tree.search_radius_vector_3d(pcd.points[query], 0.01)
np.asarray(pcd.colors)[idx[1:], :] = [0, 1, 0]
o3d.visualization.draw_plotly([pcd], width=600, height=400)

query = 5000
pcd.colors[query] = [1 , 0 , 0]
[k ,idx ,d ] = pcd_tree.search_hybrid_vector_3d( pcd.points[query],
                                                        radius =0.01,
                                                        max_nn =200 )
np.asarray(pcd.colors)[idx[1:], :] = [0 , 1 , 1]
o3d.visualization.draw_plotly([pcd], width=600, height=400)

### ICPアルゴリズム

In [ ]:
import open3d as o3d
import numpy as np
import copy


pcd1 = o3d.io.read_point_cloud( "./data/bun000.pcd" )
pcd2 = o3d.io.read_point_cloud( "./data/bun045.pcd" )

pcd_s = pcd1.voxel_down_sample(voxel_size=0.003)
pcd_t = pcd2.voxel_down_sample(voxel_size=0.003)

# 初期状態の表示
pcd_s.paint_uniform_color([0.0, 1.0, 0.0])
pcd_t.paint_uniform_color([0.0, 0.0, 1.0])
o3d.visualization.draw_plotly([pcd_s,pcd_t])

# ICPによる位置合わせ
threshold = 0.05
trans_init = np.identity(4)
obj_func = o3d.pipelines.registration.TransformationEstimationPointToPoint()
result = o3d.pipelines.registration.registration_icp( pcd_s, pcd_t,
                                                      threshold,
                                                      trans_init,
                                                      obj_func
                                                    )

trans_reg = result.transformation
print(trans_reg) 

# 得られた変換行列を点群に適用
pcd_reg = copy.deepcopy(pcd_s).transform(trans_reg)
pcd_reg.paint_uniform_color([1.0, 0.0, 0.0])
o3d.visualization.draw_plotly([pcd_reg,pcd_t])

### ICPアルゴリズムのアニメーション

リポジトリ内の`section_registration`内にNotebookがあります。

In [ ]:
%%python

import copy
import sys
import time

import numpy as np
import numpy.linalg as LA
import open3d as o3d

class ICPRegistration_PointToPoint:
    # input source, target
    def __init__( self, pcd_s, pcd_t ):
        self.pcd_s = copy.deepcopy(pcd_s)
        self.pcd_t = copy.deepcopy(pcd_t)
        self.n_points = len(self.pcd_s.points)
        
        self.pcds = []
        self.d = []
        self.closest_indices = []
        self.final_trans = np.identity(4)
        self.n_iteration = 30
        self.th_distance = 0.001
        self.th_ratio = 0.999
        
        # 初期化ステップ
        ## レジストレーションベクトル
        self.q = np.array([1.,0.,0.,0.,0.,0.,0.])
        
        self.pcd_tree = o3d.geometry.KDTreeFlann(self.pcd_t)
        self.np_pcd_t = np.asarray(self.pcd_t.points)

    def set_n_iterations( self, n ):
        self.n_iteration = n

    def set_th_distance( self, th ):
        self.th_distance = th

    def set_th_ratio( self, th ):
        self.th_ratio = th
        
    def closest_points( self ):
        idx_list = []
        distance = []
        for i in range(self.n_points):
            [k, idx, d] = self.pcd_tree.search_knn_vector_3d(self.pcd_s.points[i], 1)
            idx_list.append(idx[0])
            distance.append(d[0])

        np_pcd_y = self.np_pcd_t[idx_list]
        self.closest_indices.append( idx_list )
        self.d.append( np.sqrt(np.mean(np.array(distance))) )
        return np_pcd_y.copy()
    
    def compute_registration_param( self, np_pcd_y ):
        # 重心の取得
        mu_s = self.pcd_s.get_center()
        mu_y = np.mean(np_pcd_y, axis=0)

        # 共分散行列の計算
        np_pcd_s = np.asarray(self.pcd_s.points)
        covar = np.zeros( (3,3) )
        n_points = np_pcd_s.shape[0]
        for i in range(n_points):
            covar += np.dot( np_pcd_s[i].reshape(-1, 1), np_pcd_y[i].reshape(1, -1) )
        covar /= n_points
        covar -= np.dot( mu_s.reshape(-1,1), mu_y.reshape(1,-1) )
        
        ## anti-symmetrix matrix
        A = covar - covar.T
        delta = np.array([A[1,2],A[2,0],A[0,1]])
        tr_covar = np.trace(covar)
        i3d = np.identity(3)
        
        # symmetric matrixの作成
        Q = np.zeros((4,4))
        Q[0,0] = tr_covar
        Q[0,1:4] = delta
        Q[1:4,0] = delta
        Q[1:4,1:4] = covar + covar.T - tr_covar*i3d
        
        w, v = LA.eig(Q)
        rot = self.quaternion2rotation(v[:,np.argmax(w)])
        trans = mu_y - np.dot(rot,mu_s)
        
        self.q = np.concatenate((v[np.argmax(w)],trans))
        transform = np.identity(4)
        transform[0:3,0:3] = rot.copy()
        transform[0:3,3] = trans.copy()
        return transform
    
        
    def registration( self ):
        
        for i in range(self.n_iteration):
            # Step. 1
            np_pcd_y = self.closest_points()
            # Step. 2
            transform = self.compute_registration_param( np_pcd_y )
            # Step. 3
            self.pcd_s.transform(transform)
            self.final_trans = np.dot(transform,self.final_trans)
            self.pcds.append(copy.deepcopy(self.pcd_s))
            # Step. 4
            if ((2<i) and (self.th_ratio < self.d[-1]/self.d[-2])) or (self.d[-1] < self.th_distance):
                break
        
        return self.pcd_s
        
    # 四元数を回転行列に変換する
    def quaternion2rotation( self, q ):
        rot = np.array([[q[0]**2+q[1]**2-q[2]**2-q[3]**2, 
                         2.0*(q[1]*q[2]-q[0]*q[3]), 
                         2.0*(q[1]*q[3]+q[0]*q[2])],

                        [2.0*(q[1]*q[2]+q[0]*q[3]),
                        q[0]**2+q[2]**2-q[1]**2-q[3]**2,
                         2.0*(q[2]*q[3]-q[0]*q[1])],

                        [2.0*(q[1]*q[3]-q[0]*q[2]),
                         2.0*(q[2]*q[3]+q[0]*q[1]),
                        q[0]**2+q[3]**2-q[1]**2-q[2]**2]]
                      )
        return rot

class ICPRegistration_PointToPlane:
    # input source, target
    def __init__( self, pcd_s, pcd_t ):
        self.pcd_s = copy.deepcopy(pcd_s)
        self.pcd_t = copy.deepcopy(pcd_t)
        self.n_points = len(self.pcd_s.points)
        
        self.pcds = []
        self.d = []
        self.closest_indices = []
        self.final_trans = np.identity(4)
        self.n_iteration = 30
        self.th_distance = 0.001
        self.th_ratio = 0.999
        
        self.pcd_tree = o3d.geometry.KDTreeFlann(self.pcd_t)
        self.np_pcd_t = np.asarray(self.pcd_t.points)
        self.np_normal_t = np.asarray(self.pcd_t.normals)
    
    def set_n_iterations( self, n ):
        self.n_iterations = n
    
    def set_th_distance( self, th ):
        self.th_distance = th

    def set_th_ratio( self, th ):
        self.th_ratio = th
    
    def closest_points( self ):
        idx_list = []
        distance = []
        for i in range(self.n_points):
            [k, idx, d] = self.pcd_tree.search_knn_vector_3d(self.pcd_s.points[i], 1)
            idx_list.append(idx[0])
            distance.append(d[0])

        np_pcd_y = self.np_pcd_t[idx_list]
        np_normal_y = self.np_normal_t[idx_list]
        self.closest_indices.append( idx_list )
        self.d.append( np.sqrt(np.mean(np.array(distance))) )
        return np_pcd_y.copy(), np_normal_y.copy()
    
    def compute_registration_param( self, np_pcd_y, np_normal_y ):
        
        # A,bの計算
        np_pcd_s = np.asarray(self.pcd_s.points)
        A = np.zeros((6,6))
        b = np.zeros((6,1))
        for i in range(len(np_pcd_s)):
            xn = np.cross( np_pcd_s[i], np_normal_y[i] ) 
            xn_n = np.hstack( (xn, np_normal_y[i]) ).reshape(-1,1)
            A += np.dot( xn_n, xn_n.T )
            
            nT = np_normal_y[i].reshape(1,-1)
            p_x = (np_pcd_y[i] - np_pcd_s[i] ).reshape(-1,1)
            b += xn_n * np.dot(nT,p_x)
            
            
        u_opt = np.dot(np.linalg.inv(A),b)
        theta = np.linalg.norm(u_opt[:3])
        w = (u_opt[:3]/theta).reshape(-1)
        rot = self.axis_angle_to_matrix( w, theta )
        
        transform = np.identity(4)
        transform[0:3,0:3] = rot.copy()
        transform[0:3,3] = u_opt[3:6].reshape(-1).copy()
        
        return transform
    
    # main processing    
    def registration( self ):
        
        for i in range(self.n_iterations):
            # Step 1.
            np_pcd_y, np_normal_y = self.closest_points()
            # Step 2.
            transform = self.compute_registration_param( np_pcd_y, np_normal_y )
            # Step 3.
            self.pcd_s.transform(transform)
            self.final_trans = np.dot(transform,self.final_trans)
            self.pcds.append(copy.deepcopy(self.pcd_s))
            # Step 4.
            if ((2<i) and (self.th_ratio < self.d[-1]/self.d[-2])) or (self.d[-1] < self.th_distance):
                break
        
        return self.pcd_s
        
    def axis_angle_to_matrix( self, axis, theta ):
        """
        Args:
          axis(ndarray): rotation axis
          theta(float): rotation angle
        """

        # make skew-symmetric cross product matrix 
        w = np.array([[     0.0, -axis[2],  axis[1]],
                      [ axis[2],      0.0, -axis[0]],
                      [-axis[1],  axis[0],      0.0]
                     ])
        return np.identity(3) + (np.sin(theta)*w) + ((1-np.cos(theta))*np.dot(w,w))


def get_correspondence_lines( pcd_s, pcd_t, idx_list ):
    
    # make point cloud pair
    np_pcd_s = np.asarray(pcd_s.points)
    np_pcd_t = np.asarray(pcd_t.points)
    np_pcd_pair = np.concatenate((np_pcd_s,np_pcd_t))

    # choose pairing ids
    lines = list()
    n_points = len(pcd_s.points)
    for i in range(n_points):
        lines.append([i,n_points+idx_list[i]])

    # make lineset    
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np_pcd_pair),
        lines=o3d.utility.Vector2iVector(lines),
    )
    return line_set

def visualize_icp_progress( reg ):

    pcds = reg.pcds
    indices = reg.closest_indices
    pcd_t = reg.pcd_t

    cnt = 0
    pcd = o3d.geometry.PointCloud()
    line_set = o3d.geometry.LineSet()
    def reg( vis ):
        nonlocal cnt
        
        pcd.points = pcds[cnt].points
        pcd.paint_uniform_color([1.0,0.0,0.0])
        lines = get_correspondence_lines( pcds[cnt], pcd_t, indices[cnt] )
        line_set.lines = lines.lines
        line_set.points = lines.points
        vis.update_geometry( pcd )
        vis.update_geometry( line_set )
        
        cnt+=1
        time.sleep(0.3)
        if  len(pcds)-1 <= cnt:
            cnt = 0
    
    o3d.visualization.draw_geometries_with_animation_callback([pcd, pcd_t, line_set], 
                                                              reg, width=640, height=500)


mode = 0  # 0はPoint-to-Point, 1はPoint-to-Plane
pcd1 = o3d.io.read_point_cloud( "./data/bun000.pcd" )
pcd2 = o3d.io.read_point_cloud( "./data/bun045.pcd" )

pcd_s = pcd1.voxel_down_sample(voxel_size=0.003)
pcd_t = pcd2.voxel_down_sample(voxel_size=0.003)

# 初期状態の表示
pcd_s.paint_uniform_color([0.0, 1.0, 0.0])
pcd_t.paint_uniform_color([0.0, 0.0, 1.0])
o3d.visualization.draw_geometries([pcd_t, pcd_s] )

# ICPの実行．
reg = ICPRegistration_PointToPoint(pcd_s, pcd_t)
if mode == 1:
	print("Run Point-to-plane ICP algorithm")
	reg = ICPRegistration_PointToPlane(pcd_s, pcd_t)
reg.set_th_distance( 0.003 )
reg.set_n_iterations( 100 )
reg.set_th_ratio( 0.999 )
pcd_reg = reg.registration()

print("# of iterations: ", len(reg.d))
print("Registration error [m/pts.]:", reg.d[-1] )
print("Final transformation \n", reg.final_trans )

pcd_reg.paint_uniform_color([1.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_t, pcd_reg] )

# ICPの実行の様子の可視化
visualize_icp_progress( reg )

## 点群からの物体認識

RGB-D Object Datasetのダウンロードと最近傍法による分類

In [ ]:
import os
import open3d as o3d
import numpy as np

dirname = "rgbd-dataset"
classes = ["apple", "banana", "camera"]
url="https://rgbd-dataset.cs.washington.edu/dataset/rgbd-dataset_pcd_ascii/"
for i in range(len(classes)):
    if not os.path.exists(dirname + "/" + classes[i]):
        os.system("wget " + url + classes[i] + "_1.tar")
        os.system("tar xvf " + classes[i] + "_1.tar")


def extract_fpfh( filename ):
    print (" ", filename)
    pcd = o3d.io.read_point_cloud(filename)
    pcd = pcd.voxel_down_sample(0.01)
    pcd.estimate_normals(
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=10))
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd,
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100))
    sum_fpfh = np.sum(np.array(fpfh.data),1)
    return( sum_fpfh / np.linalg.norm(sum_fpfh) )


# Extract features
nsamp = 100
feat_train = np.zeros( (len(classes), nsamp, 33) )
feat_test = np.zeros( (len(classes), nsamp, 33) )
for i in range(len(classes)):
    print ("Extracting train features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_1_" + str(n+1) + ".pcd"
        feat_train[ i, n ] = extract_fpfh( filename )
    print ("Extracting test features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_4_" + str(n+1) + ".pcd"
        feat_test[ i, n ] = extract_fpfh( filename )


# 1-NN classification
for i in range(len(classes)):
    max_sim = np.zeros((3, nsamp))
    for j in range(len(classes)):
        sim = np.dot(feat_test[i], feat_train[j].transpose())
        max_sim[j] = np.max(sim,1)
    correct_num = (np.argmax(max_sim,0) == i).sum()
    print ("Accuracy of", classes[i], ":", correct_num*100/nsamp, "%")


### 特徴点マッチング

In [ ]:
%%python

import sys
import numpy as np
import copy
import open3d as o3d


def create_lineset_from_correspondences( corrs_set, pcd1, pcd2, 
                                         transformation=np.identity(4) ):
    """ 対応点セットからo3d.geometry.LineSetを作成する．
    Args:
        result(o3d.utility.Vector2iVector) ): 対応点のidセット
        pcd1(o3d.geometry.PointCloud): resultを計算した時の点群1
        pcd2(o3d.geometry.PointCloud): resultを計算した時の点群2
        transformation(numpy.ndarray): 姿勢変換行列(4x4)
    Return:
        o3d.geometry.LineSet
    """
    pcd1_temp = copy.deepcopy(pcd1)
    pcd1_temp.transform(transformation) 
    corrs = np.asarray(corrs_set)
    np_points1 = np.array(pcd1_temp.points)
    np_points2 = np.array(pcd2.points)
    points = list()
    lines = list()

    for i in range(corrs.shape[0]):
        points.append( np_points1[corrs[i,0]] )
        points.append( np_points2[corrs[i,1]] )
        lines.append([2*i, (2*i)+1])

    colors = [np.random.rand(3) for i in range(len(lines))]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set


def draw_registration_result( source, target, transformation ):
    """ sourceをtransformationによって姿勢変換してtargetと一緒に表示する
    Args:
        source(list[o3d.geometry.PointCloud]): 点群（姿勢変換する）
        target(list[o3d.geometry.PointCloud]): 点群（そのまま表示）
        transformation(numpy.ndarray): 姿勢変換行列(4x4)
    """
    pcds = list()
    for s in source: 
        temp = copy.deepcopy(s)
        pcds.append( temp.transform(transformation) )
    pcds += target
    o3d.visualization.draw_geometries(pcds, zoom=0.3199,
			                          front = [0.024, -0.225, -0.973],
			                          lookat = [0.488, 1.722, 1.556],
			                          up = [0.047, -0.972, 0.226])


def keypoint_and_feature_extraction( pcd, voxel_size ):

	keypoints = pcd.voxel_down_sample(voxel_size)

	viewpoint = np.array([0.,0.,0.], dtype='float64')
	radius_normal = 2.0*voxel_size
	keypoints.estimate_normals(
		o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
	keypoints.orient_normals_towards_camera_location( viewpoint )

	radius_feature = 5.0*voxel_size
	feature = o3d.pipelines.registration.compute_fpfh_feature(
        keypoints,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
	return keypoints, feature


if __name__ == "__main__":
    #データ読み込み
    filename1 = './3rdparty/Open3D/examples/test_data/ICP/cloud_bin_0.pcd'
    filename2 = './3rdparty/Open3D/examples/test_data/ICP/cloud_bin_1.pcd'
    source = o3d.io.read_point_cloud(filename1)
    target = o3d.io.read_point_cloud(filename2)

    source.paint_uniform_color([0.5,0.5,1])
    target.paint_uniform_color([1,0.5,0.5])
    initial_trans = np.identity(4)
    initial_trans[0,3] = -3.0
    draw_registration_result([source], [target], initial_trans)

    # 特徴量記述
    voxel_size = 0.1
    s_kp, s_feature = keypoint_and_feature_extraction( source, voxel_size )
    t_kp, t_feature = keypoint_and_feature_extraction( target, voxel_size )

    s_kp.paint_uniform_color([0,1,0])
    t_kp.paint_uniform_color([0,1,0])
    draw_registration_result([source,s_kp], [target,t_kp], initial_trans)

    np_s_feature = s_feature.data.T
    np_t_feature = t_feature.data.T

    # 対応点探索
    corrs = o3d.utility.Vector2iVector()
    threshold = 0.9
    for i,feat in enumerate(np_s_feature):
        # source側の特定の特徴量とtarget側の全特徴量間のノルムを計算
        distance = np.linalg.norm( np_t_feature - feat, axis=1 )
        nearest_idx = np.argmin(distance)
        dist_order = np.argsort(distance)
        ratio = distance[dist_order[0]] / distance[dist_order[1]]
        if ratio < threshold:
            corr = np.array( [[i],[nearest_idx]], np.int32 )
            corrs.append( corr )

    print('対応点セットの数：', (len(corrs)) )


    # 姿勢計算
    ## 全点利用
    print('全点利用による姿勢計算' )
    line_set = create_lineset_from_correspondences( corrs, s_kp, t_kp, initial_trans )
    draw_registration_result([source,s_kp], [target,t_kp, line_set], initial_trans)

    trans_ptp = o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
    trans_all = trans_ptp.compute_transformation( s_kp, t_kp, corrs )
    draw_registration_result([source], [target], trans_all )

    ## RANSAC利用
    print('RANSACによる姿勢計算' )
    draw_registration_result([source,s_kp], [target,t_kp, line_set], initial_trans)
    distance_threshold = voxel_size*1.5
    result = o3d.pipelines.registration.registration_ransac_based_on_correspondence(
            s_kp, t_kp, corrs,
            distance_threshold,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
            ransac_n = 3, 
            checkers = [
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
            ], 
            criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
            )

    line_set = create_lineset_from_correspondences( result.correspondence_set, s_kp, t_kp, initial_trans )
    draw_registration_result([source], [target], result.transformation)

### プリミティブ検出
####  平面の検出

In [ ]:
import numpy as np
import copy

import open3d as o3d

pcd = o3d.io.read_point_cloud("./data/tabletop_scene.ply")
o3d.visualization.draw_plotly([pcd], 
                                  zoom=0.459,
			                      front=[ 0.349, 0.063, -0.934 ],
			                      lookat=[ 0.039, 0.007, 0.524 ],
			                      up=[ -0.316, -0.930, -0.181 ]
                                  )

plane_model, inliers = pcd.segment_plane(distance_threshold=0.005,
                                         ransac_n=3,
                                         num_iterations=500)

[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

plane_cloud = pcd.select_by_index(inliers)
plane_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)

o3d.visualization.draw_plotly([plane_cloud, outlier_cloud],
                                  zoom=0.459,
			                      front=[ 0.349, 0.063, -0.934 ],
			                      lookat=[ 0.039, 0.007, 0.524 ],
			                      up=[ -0.316, -0.930, -0.181 ]
                                  )

####  球の検出

In [ ]:
import numpy as np
import copy

import open3d as o3d

def ComputeSphereCoefficient( p0, p1, p2, p3 ):
    """ 与えられた4点を通る球の方程式のパラメータ(a,b,c,r)を出力する．
        解が求まらない場合は，
    Args:
      p0,p1,p2,p3(numpy.ndarray): 4 points (x,y,z)
    Return:
      Sphere coefficients.
    """

    A = np.array([p0-p3,p1-p3,p2-p3])
    
    p3_2 = np.dot(p3,p3)
    b = np.array([(np.dot(p0,p0)-p3_2)/2,
                  (np.dot(p1,p1)-p3_2)/2,
                  (np.dot(p2,p2)-p3_2)/2])
    coeff = np.zeros(3)
    try:
        ans = np.linalg.solve(A,b)
    except:
        print( "!!Error!! Matrix rank is", np.linalg.matrix_rank(A) )
        print( "  Return", coeff )
        pass
    else:
        tmp = p0-ans
        r = np.sqrt( np.dot(tmp,tmp) )
        coeff = np.append(ans,r)

    return coeff

def EvaluateSphereCoefficient( pcd, coeff, distance_th=0.01 ):
    """ 球の方程式の係数の当てはまりの良さを評価する．
    Args:
      pcd(numpy.ndarray): Nx3 points
      coeff(numpy.ndarray): shpere coefficient. (a,b,c,r)
      distance_th(float):
    Returns:
      fitness: score [0-1]. larger is better
      inlier_dist: smaller is better
      inliers: indices of inliers
    """
    fitness = 0 # インライア点数/全点数
    inlier_dist = 0 #インライアの平均距離
    inliers = None #インライア点の番号セット
    
    dist = np.abs( np.linalg.norm( pcd - coeff[:3], axis=1 ) - coeff[3] )
    n_inlier = np.sum(dist<distance_th)
    if n_inlier != 0:
        fitness = n_inlier / pcd.shape[0]
        inlier_dist = np.sum((dist<distance_th)*dist)/n_inlier
        inliers = np.where(dist<distance_th)[0]
    
    return fitness, inlier_dist, inliers


if __name__ == "__main__":
    # データ読み込み
    pcd = o3d.io.read_point_cloud("./data/tabletop_scene_segment.ply")
    np_pcd = np.asarray(pcd.points)
    o3d.visualization.draw_plotly([pcd])

    # Parameters
    ransac_n = 4 # 点群から選択する点数．球の場合は4．
    num_iterations = 1000 # RANSACの試行回数
    distance_th = 0.005 # モデルと点群の距離のしきい値
    max_radius = 0.05 # 検出する球の半径の最大値

    # 解の初期化
    best_fitness = 0 # モデルの当てはめの良さ．インライア点数/全点数
    best_inlier_dist = 10000.0 #インライア点の平均距離
    best_inliers = None # 元の点群におけるインライアのインデクス
    best_coeff = np.zeros(4) # モデルパラメータ

    for n in range(num_iterations):
        c_id = np.random.choice( np_pcd.shape[0], 4, replace=False )
        coeff = ComputeSphereCoefficient( np_pcd[c_id[0]], np_pcd[c_id[1]], np_pcd[c_id[2]], np_pcd[c_id[3]] )
        if max_radius < coeff[3]:
            continue
        fitness, inlier_dist, inliers = EvaluateSphereCoefficient( np_pcd, coeff, distance_th )
        if (best_fitness < fitness) or ((best_fitness == fitness) and (inlier_dist<best_inlier_dist)):
            best_fitness = fitness
            best_inlier_dist = inlier_dist
            best_inliers = inliers
            best_coeff = coeff
            print(f"Update: Fitness = {best_fitness:.4f}, Inlier_dist = {best_inlier_dist:.4f}")

    if best_coeff.any() != False:
        print(f"Sphere equation: (x-{best_coeff[0]:.2f})^2 + (y-{best_coeff[1]:.2f})^2 + (z-{best_coeff[2]:.2f})^2 = {best_coeff[3]:.2f}^2")
    else:
        print("No sphere detected.")

    # 結果の可視化
    sphere_cloud = pcd.select_by_index(best_inliers)
    sphere_cloud.paint_uniform_color([0, 0, 1.0])
    outlier_cloud = pcd.select_by_index(best_inliers, invert=True)

    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=best_coeff[3])
    mesh_sphere.compute_vertex_normals()
    mesh_sphere.paint_uniform_color([0.3, 0.3, 0.7])
    mesh_sphere.translate(best_coeff[:3])
    o3d.visualization.draw_plotly([mesh_sphere]+[sphere_cloud+outlier_cloud])

### セグメンテーション(DBSCAN)

In [ ]:
import sys
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

filename = './3rdparty/Open3D/examples/test_data/fragment.pcd'
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

labels = np.array(pcd.cluster_dbscan(eps=0.02, min_points=10))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / max(max_label,1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_plotly([pcd], zoom=0.8,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])


6章以降は`section_deep_learning`、`section_other_types`内にNotebookがあるのでそちらを実行。